<img src="logos/‎cover‎001.png" alt="Drawing"/>

In [1]:
# imports
import pyxdf
import pandas as pd
import datetime
import os
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
import seaborn as sns

# Latency tests
Situation:
We designed a virtual environment in Unity that indefinitely switches the color of a canvas from black -> white -> gray -> black. The test files with different durations are stored in the folder "data". Before loading them, we want to access their path and save them in a dictionary.

### Task 1:
- Get all files from "data" folder, sort them alphabetically and save them to a variable "files"
Hint: os.listdir()
Desired output: ['lsl_test1.xdf', 'lsl_test2.xdf', 'lsl_test3.xdf']

In [ ]:
# get all files from the folder "data"
files =
# sort them alphabetically


In [ ]:
# save all files to an empty dictionary "recordings"
recordings = {}

for i, file in enumerate(files):  # store and display all files
    created = os.path.getmtime(f"data/{file}")  # creation timestamp
    created = datetime.datetime.fromtimestamp(created)  # translate as datetime
    created = created.strftime("%d.%m.%Y %H:%M")  # arrange it
    recordings[i] = {"file": file, "created": created}

files = [f.split(".")[0] for f in files]
# display the recordings and metadata
display(recordings)

# Extensible Data Format (XDF)

.XDF is a format to store multiple channels of time series data with specific meta information.
For the latency test, we stored three streams
- Unity: 'Visual',
- EEG: 'openvibeMarkers', 'openvibeSignal'

When loaded, the .XDF file results in a list of dictionaries with some components:
1. **'info':**
2. **'time_series':** the information sent from Unity and EEG to via LSL
3. **'time_stamps':** the time stamps for each datapoint received

### Task 2:
Load data for recording one in a variable _streams_ and have a look at structure.
<strong>Hint:</strong>  for more information on how to load and what is actually loaded from .XDF, visit [.XDF website info](https://pypi.org/project/pyxdf/)

In [ ]:
# load data


### Accessing streams
Once we load the streams, we can access each list in the dictionary by indicating, for example, the component we want to access such as a specific stream recorded (an index), 'info', 'time_stamps', 'time_series', etc.
For specific information:
- streams[2]['info']['name']

For a particular stream component:
- streams[2]['time_stamps']

For a specific eeg channel
- streams[2]["time_series"][:,64]

In [ ]:
# access the second stream's time_stamps
streams[2]['time_stamps']

In [ ]:
# acess all stream names and index in stream list
s_names = {streams[i]["info"]["name"][0]: i for i in range(len(streams))}
s_names

### Task 3: Defining functions

<strong>Function 1</strong>
Define a function that given the .xdf streams, returns the relevant streams we recorded:
- The Unity samples were sent via a stream called _Visual_
- The EEG light sensor data was sent via a stream called _openvibeSignal_

In [ ]:
# define a function to select "Visual streams  and openvibeSignal from streams
def select_streams(streams):


In [ ]:
# use the function to select the Unity ('u_ch') and EEG ('e_ch') streams


<strong>Function 2</strong>

Define a function that returns the computer "hostname" from the "info" dictionary given the streams data and the selected streams

In [ ]:
# retrieve computer hostname
def stream_host(streams, u_ch, e_ch):


<strong>Function 3</strong>

Define a function that returns the "time_stamps" for each selected stream in the streams' data.

Notice that LSL assigns a random large number at the start of the recording to both EEG and Unity to synchronize them.

In [ ]:
# retrieve Unity ('u') and EEG ('e') time stamps
def streams_ts(streams, u_ch, e_ch):


<strong>Function 4</strong>

Define a function that corrects the Unity and EEG timestamps to start at zero

<strong>Hint:</strong>  return a numpy array containing all corrected timestamps

In [ ]:
# correct Unity ('u') and EEG ('e') time stamps to start at 0
def corrected_ts(time_stamps):


<strong>Function 5</strong>

Define a function that calculates the latency of one sample to the next using the normalized timestamps.

<strong>Hint:</strong> latency defined as the time difference between a timestamp and the next one

In [ ]:
# calculate the latency between one sample and the next
def calculate_latency(ts_norm):


<strong>Function 6</strong>

Define a function that retrieves the EEG and Unity streams data stored in _"time_series"_
Hint: "time_series" contains the data sample that corresponds to each timestamp.

For Unity, we saved 0, 1, and 2 every time it switched from black to gray to white (channel 1)
EEG contains the changes in light intensity for each of the colors collected in channel 64

In [ ]:
# retrieve Unity ('u') and EEG ('e') stream data
def streams_data(streams, u_ch, e_ch):


<strong>Function 7</strong>

Define a function that calculates the test duration in minutes.

In [ ]:
# calculate recording's duration
def test_duration(time_stamps):


<strong>Function 8</strong>

Define a function that calculates the average sampling rate


In [ ]:
# calculate sampling rate (fps)
def sampling_fps(t_duration, time_stamps):


# Access all tests data

Now we can use the above functions to iterate over all the recordings and retrieve the corresponding data.

In [ ]:
# extract streams info for all files
overview_df = pd.DataFrame()
recordings_data_eeg = pd.DataFrame()
recordings_data_unity = pd.DataFrame()

for r in recordings:
    # current filename
    file = recordings[r]["file"]
    # load data
    streams, _ = pyxdf.load_xdf(f"data/{file}")
    # select stream channels
    u_ch, e_ch = select_streams(streams)
    # computer host name
    u_host, e_host = stream_host(streams, u_ch, e_ch)
    # select timestamps for each stream
    u_ts, e_ts = streams_ts(streams, u_ch, e_ch)
    # corrected timestamps to start at zero
    u_ts_corrected = corrected_ts(u_ts)
    e_ts_corrected = corrected_ts(e_ts)
    # latency of normalized timestamps
    u_latency = calculate_latency(u_ts_corrected)
    e_latency = calculate_latency(e_ts_corrected)
    # samples data from streams
    u_data, e_data = streams_data(streams, u_ch, e_ch)
    # calculate Unity - eeg time difference at start of recording
    # in milliseconds
    diff_ts = (u_ts[0] - e_ts[0]) * 1000
    # calculate recoding duration
    unity_duration =  test_duration(u_ts)
    eeg_duration =  test_duration(e_ts)
    # calculate sampling rate (FPS)
    unity_sr = sampling_fps(unity_duration, u_ts)
    eeg_sr = sampling_fps(eeg_duration, e_ts)
    # Store overview statistics for each test
    overview_df = pd.concat([overview_df, pd.DataFrame.from_dict({"file_name": file,
                         "u_duration": unity_duration,
                         "diff_ts": diff_ts,
                         "eeg_duration": eeg_duration,
                         "u_fps": unity_sr,
                         "eeg_fps": eeg_sr,
                         "u_host": u_host,
                         "eeg_host": e_host})])
    # Store key stream for use in analysis
    recording_data_eeg = pd.concat([pd.DataFrame(np.resize(file, len(e_ts_corrected)), columns=['filename']),
                               pd.DataFrame(e_ts, columns=['e_ts']),
                               pd.DataFrame(e_ts_corrected, columns=['e_ts_corrected']),
                               pd.DataFrame(e_latency, columns=['e_latency']),
                               pd.DataFrame(e_data, columns=['sensor_data'])], axis=1)
    recording_data_unity = pd.concat([pd.DataFrame(np.resize(file, len(u_ts_corrected)), columns=['filename']),
                               pd.DataFrame(u_ts, columns=['u_ts']),
                               pd.DataFrame(u_ts_corrected, columns=['u_ts_corrected']),
                               pd.DataFrame(u_latency, columns=['u_latency']),
                               pd.DataFrame(u_data, columns=['switch_state'])], axis=1)
    recordings_data_eeg = pd.concat([recordings_data_eeg, recording_data_eeg])
    recordings_data_unity = pd.concat([recordings_data_unity, recording_data_unity])

In [ ]:
# preview recordings' information
overview_df

In [ ]:
# preview of Unity data
recordings_data_unity

In [ ]:
# preview EEG data
recordings_data_eeg

# Tests' descriptive statistic
Now we can calculate the average latency with which each Unity and EEG sample entered the system for each of the recorded tests.

### Task 4:
- Calculate the average latency for each device (EEG, Unity)
- How constant are the frame rates?

In [ ]:
# Unity's latency


In [ ]:
# EEG's latency


# Latencies visualization
### Task 5:
- Visualize the latencies distributions for EEG and Unity

In [ ]:
# start latencies figure/subplots here


# Markers' visualizations
Now we will visualize the behavior between EEG and Unity for the first 5 seconds of recording in one of the tests.
### Task 6:
- Select test number 3 from the Unity and EEG dataframe and visualize the first 5 seconds of recording

In [ ]:
# Unity's first 5 seconds

# EEG's first 5 seconds


In [ ]:
# start recording figure/subplots here


# Conclusions:
1. EEG sampling rate was extremely constant with each sample latency (mean = ~ 0.98 ms, std = 0.0)
2. Unity sampling rate was constant with each sample latency (mean = ~ 11.11 ms, std = 0.75 ms)
3. Latencies for both devices were constant independently of recording duration
4. <strong>LSL is a reliable method to collect brain and behavioral data when combining EEG and VR.</strong>